In [1]:
import math
from browser_libs import get_collection_items, get_collection_names, get_collection_for_user
from memoize import memoize # pip install memoize2
from collections import Counter
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
collection_names = get_collection_names()
for x in collection_names:
  if 'experiment_vars' in x and 'for_goal' not in x:
    print(x)
    break

960c17bec89c59404248b47d_synced:experiment_vars


In [3]:
@memoize
def get_users_with_choose_difficulty():
  collection_names = get_collection_names()
  output = []
  for x in collection_names:
    if x.endswith('_internal:choose_difficulty'):
      user = x.replace('_internal:choose_difficulty', '')
      output.append(user)
  return output


In [4]:
@memoize
def get_choose_difficulty_items_for_user(user):
  collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
  output = []
  for x in collection_items:
    if 'is_new_session' not in x:
      continue
    if x['is_new_session'] != True:
      continue
    if ('developer_mode' in x) and (x['developer_mode'] == True):
      continue
    if ('is_preview_mode' in x) and (x['is_preview_mode'] == True):
      continue
    if ('is_suggestion_mode' in x) and (x['is_suggestion_mode'] == True):
      continue
    output.append(x)
  return output

In [5]:
def get_choose_difficulty_counts_for_user(user):
  collection_items = get_choose_difficulty_items_for_user(user)
  output = {}
  for x in collection_items:
    if 'type' not in x:
      continue
    if x['type'] != 'action':
      continue
    if 'difficulty' not in x:
      continue
    difficulty = x['difficulty']
    if difficulty not in output:
      output[difficulty] = 1
    else:
      output[difficulty] += 1
  return output

In [6]:
def get_choose_difficulty_counts_for_user_list(user_list):
  output = Counter()
  for user in user_list:
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    for difficulty,count in difficulty_counts.items():
      output[difficulty] += count
  return output

In [7]:
def get_choose_difficulty_counts_for_user_list_user_normalized(user_list):
  output = Counter()
  for user in user_list:
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    difficulty_counts = to_percent_dict(difficulty_counts)
    for difficulty,count in difficulty_counts.items():
      output[difficulty] += count
  return to_percent_dict(output)

In [8]:
def get_choose_difficulty_counts_for_all_users():
  user_list = get_users_with_choose_difficulty()
  return get_choose_difficulty_counts_for_user_list(user_list)

In [9]:
def get_total_counts_for_user(user):
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  return sum(difficulty_counts.values())

In [10]:
def num_types_tried(user):
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  return len(difficulty_counts.keys())

In [11]:
def did_user_try_multiple(user):
  return num_types_tried(user) > 1

In [12]:
def get_try_multiple_counts_for_all_users():
  output = {
    'none_total': 0,
    'one_total': 0,
    'one_tried': 0,
    'multiple_tried': 0,
  }
  for user in get_users_with_choose_difficulty():
    total_counts = get_total_counts_for_user(user)
    if total_counts == 0:
      output['none_total'] += 1
      continue
    if total_counts == 1:
      output['one_total'] += 1
      continue
    if did_user_try_multiple(user):
      output['multiple_tried'] += 1
    else:
      output['one_tried'] += 1
  return output

In [13]:
def get_breakdown_for_one_tried():
  output = Counter()
  for user in get_users_with_choose_difficulty():
    total_counts = get_total_counts_for_user(user)
    if total_counts <= 1:
      continue
    if did_user_try_multiple(user):
      continue
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    chosen_difficulty = list(difficulty_counts.keys())[0]
    output[chosen_difficulty] += difficulty_counts[chosen_difficulty]
  return output

In [14]:
print(len(get_users_with_choose_difficulty()))

1294


In [15]:
print(get_choose_difficulty_counts_for_all_users())

Counter({'nothing': 29566, 'easy': 12809, 'medium': 6481, 'hard': 2723})


In [16]:
print(get_try_multiple_counts_for_all_users())

{'none_total': 342, 'one_total': 155, 'one_tried': 207, 'multiple_tried': 590}


In [17]:
print(get_breakdown_for_one_tried())

Counter({'nothing': 2860, 'easy': 696, 'medium': 509, 'hard': 52})


In [18]:
@memoize
def get_abtest_settings(user):
  output = {}
  collection_items = get_collection_for_user(user, 'synced:experiment_vars')
  for item in collection_items:
    if 'key' not in item:
      continue
    if 'val' not in item:
      continue
    key = item['key']
    val = item['val']
    output[key] = val
  return output


In [19]:
def get_abtest_options_for_group(user_list):
  conditions = {}
  for user in user_list:
    abtest_settings = get_abtest_settings(user)
    for k,v in abtest_settings.items():
      if k == 'intervention_firstimpression_notice_seenlist':
        continue
      if k not in conditions:
        conditions[k] = []
      if v not in conditions[k]:
        conditions[k].append(v)
  output = {}
  for abtest_name,options in conditions.items():
    if len(options) > 1:
      output[abtest_name] = options
  return output

In [20]:
def get_abtest_condition_to_user_list(abtest_name):
  # note this only applies to users in the get_users_with_choose_difficulty experiment currently
  output = {}
  for user in get_users_with_choose_difficulty():
    abtest_settings = get_abtest_settings(user)
    if abtest_name not in abtest_settings:
      continue
    abtest_option = abtest_settings[abtest_name]
    if abtest_option not in output:
      output[abtest_option] = []
    output[abtest_option].append(user)
  return output

In [21]:
difficulty_interface_to_users = get_abtest_condition_to_user_list('choose_difficulty_interface')

In [22]:
print(difficulty_interface_to_users)

{'this_intervention_toast': ['e0ea34c81d4b50cddc7bd752', '6bbc8f6c01b33427ff8f20e9', '4d99e8cdfa7fe3d533b9745f', '6751d267a976e214e678b276', '350240ac1aed84fc034ef0d9', '12ceeebc951892700938281f'], 'this_intervention': ['6e055258e3625ab1db57f0fa', 'cc43e1c331c9d74bf7b00fc6', '06b54759c28de7967024cedf', '0812df1382381cee3edab4b6', 'de6a908835401dcaf037cc95', '6ffae4b0e0a58695ae4ee174', '9ee9b7534c9bcafec6a13773', 'cf2b84b289d13a0fe56a7f71', '9ba55c3765b5fcfdb1753a1b', '329cbe941cea398f04cd560f', '39987e39f3e03b1dfc68bd5c', 'f05959c1a2ccd135e93b531e', '7d33ed1787c5d8b4f5bb9b11', '0951f6da6eff18c10db17810', 'b1cd523a3e8b9c65ed79dd9c', '6326707eff8b6e94a81fcc28', '028033fdfa87801f5ab9b713', '9c5f2b25a366062490398273', '4893b81edd7f0e2b102b0d1a', '9add59262ea1f3f7619decda', '97c51558e4074650392bf084', '1bfb8dc0cc5a41097c166a60', '4f9dfd154111179c642c8cf9', '83a0c03f7605c80cf2c395fa', '7bd50164fe03fd6254784ed3', '007c24675bd2b9078cc3cce5', '8fdc79a87aa36a0cfaa325e9', '025cf2acf9559abfe3e4fb2

In [23]:
frequency_of_choose_difficulty_to_users = get_abtest_condition_to_user_list('frequency_of_choose_difficulty')

In [24]:
def get_choose_difficulty_level_mean_for_user_list(users):
  difficulty_to_counts = get_choose_difficulty_counts_for_user_list(users)
  difficulty_to_value = {
    'nothing': 0,
    'easy': 1,
    'medium': 2,
    'hard': 3,
  }
  values = []
  for difficulty,count in difficulty_to_counts.items():
    value = difficulty_to_value[difficulty]
    values.append(value)
  return np.mean(values)

In [25]:
def get_choose_difficulty_level_mean_by_abtest(abtest_name):
  condition_to_user_list = get_abtest_condition_to_user_list(abtest_name)
  for condition,user_list in condition_to_user_list.items():
    print(condition + ':' + str(get_choose_difficulty_level_mean_for_user_list(user_list)))

In [26]:
def get_key_to_ordering_mappings():
  key_orderings = [
    [
      'this_intervention',
      'time_afford',
      'settings_update',
    ],
    [
      'nothing',
      'easy',
      'medium',
      'hard',
    ],
  ]
  key_to_ordering = {}
  for ordering in key_orderings:
    key = ' '.join(sorted(ordering))
    key_to_ordering[key] = ordering
  return key_to_ordering

def order_list(keys):
  key = ' '.join(sorted(keys))
  key_to_ordering_mappings = get_key_to_ordering_mappings()
  return get_key_to_ordering_mappings()[key]

def printdict(d):
  keys = order_list(d.keys())
  for x in keys:
    print(x + ': ' + str(d[x]))

def to_percent_dict(d):
  output = {}
  total = sum(d.values())
  for k,v in d.items():
    output[k] = v / total
  return output

def printdict_percent(d):
  d = to_percent_dict(d)
  printdict(d)

In [27]:
def plotbar(values, labels=None, title=''):
  data = [go.Bar(
    x=labels,
    y=values,
  )]
  layout = go.Layout(title=title)
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)

def plotbarh(values, labels=None, title=''):
  data = [go.Bar(
    y=labels,
    x=values,
    orientation='h',
  )]
  layout = go.Layout(title=title)
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)

def plothist(values, title=''):
  data = [go.Histogram(x=values)]
  layout = go.Layout(title=title)
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)


In [28]:
def plotdict(d, title=''):
  keys = order_list(d.keys())
  values = [d[k] for k in keys]
  plotbarh(values, keys, title)

In [29]:
#for condition,user_list in condition_to_user_list.items():
#  print(condition + ':' + str(get_choose_difficulty_level_mean_for_user_list(user_list)))

In [30]:
get_choose_difficulty_level_mean_by_abtest('choose_difficulty_interface')

this_intervention_toast:1.0
this_intervention:1.5
time_afford:1.5
settings_update:1.5


In [31]:
get_choose_difficulty_level_mean_by_abtest('frequency_of_choose_difficulty')

0.5:1.5
1.0:1.5
0.0:1.5
0.25:1.5


In [32]:
printdict_percent(get_choose_difficulty_counts_for_user_list(difficulty_interface_to_users['time_afford']))

nothing: 0.3984375
easy: 0.17578125
medium: 0.33642578125
hard: 0.08935546875


In [33]:
printdict_percent(get_choose_difficulty_counts_for_user_list(difficulty_interface_to_users['this_intervention']))

nothing: 0.42726621282694377
easy: 0.34163382300250805
medium: 0.16911501254030814
hard: 0.06198495163024006


In [34]:
printdict_percent(get_choose_difficulty_counts_for_user_list(difficulty_interface_to_users['settings_update']))

nothing: 0.41920785138450756
easy: 0.19348054679284962
medium: 0.26498422712933756
hard: 0.12232737469330529


In [35]:
printdict(get_choose_difficulty_counts_for_user_list_user_normalized(difficulty_interface_to_users['time_afford']))

nothing: 0.40309513636068917
easy: 0.17678153289737641
medium: 0.3378533883500458
hard: 0.08226994239188848


In [36]:
printdict(get_choose_difficulty_counts_for_user_list_user_normalized(difficulty_interface_to_users['this_intervention']))

nothing: 0.390813813079382
easy: 0.3391801336802089
medium: 0.18678554547896845
hard: 0.08322050776144051


In [37]:
printdict(get_choose_difficulty_counts_for_user_list_user_normalized(difficulty_interface_to_users['settings_update']))

nothing: 0.4881834049538276
easy: 0.26681993870982706
medium: 0.1898966053234261
hard: 0.055100051012919184


In [38]:
plotdict(get_choose_difficulty_counts_for_user_list_user_normalized(difficulty_interface_to_users['time_afford']), 'Difficulty selections for "time you can afford this visit" interface')

In [39]:
plotdict(get_choose_difficulty_counts_for_user_list_user_normalized(difficulty_interface_to_users['this_intervention']), 'Difficulty selections for "intervention difficulty you want this visit" interface')

In [40]:
plotdict(get_choose_difficulty_counts_for_user_list_user_normalized(difficulty_interface_to_users['settings_update']), 'Difficulty selections for "update your difficulty settings" interface')

In [42]:
plotdict(get_choose_difficulty_counts_for_user_list_user_normalized(frequency_of_choose_difficulty_to_users['1.0']), 'Difficulty selections if selection interface shown with p=1.0')

In [43]:
plotdict(get_choose_difficulty_counts_for_user_list_user_normalized(frequency_of_choose_difficulty_to_users['0.5']), 'Difficulty selections if selection interface shown with p=0.5')

In [44]:
plotdict(get_choose_difficulty_counts_for_user_list_user_normalized(frequency_of_choose_difficulty_to_users['0.25']), 'Difficulty selections if selection interface shown with p=0.25')

In [47]:
def get_daynum_to_difficulty_choices(user):
  difficulty_choices = get_choose_difficulty_items_for_user(user)
  first_timestamp = None
  output = {}
  for item in difficulty_choices:
    if 'type' not in item:
      continue
    if item['type'] != 'action':
      continue
    if 'difficulty' not in item:
      continue
    difficulty = item['difficulty']
    timestamp = item['timestamp_local']
    if first_timestamp == None or timestamp < first_timestamp:
      first_timestamp = timestamp
  for item in difficulty_choices:
    if 'type' not in item:
      continue
    if item['type'] != 'action':
      continue
    if 'difficulty' not in item:
      continue
    difficulty = item['difficulty']
    timestamp = item['timestamp_local']
    daynum = (timestamp - first_timestamp) / (1000 * 3600 * 24)
    daynum = int(math.floor(daynum))
    if daynum not in output:
      output[daynum] = {}
    if difficulty not in output[daynum]:
      output[daynum][difficulty] = 0
    output[daynum][difficulty] += 1
  return output

def get_user_to_daynum_to_difficulty_choices():
  output = {}
  user_list = get_users_with_choose_difficulty()
  for user in user_list:
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    if len(difficulty_counts.keys()) == 0:
      continue
    daynum_to_difficulty_choices = get_daynum_to_difficulty_choices(user)
    if len(daynum_to_difficulty_choices.keys()) == 0:
      continue
    output[user] = daynum_to_difficulty_choices
  return output

def get_daynum_to_difficulty_choices_over_n_days(num_days):
  user_to_daynum_to_difficulty_choices = get_user_to_daynum_to_difficulty_choices()
  output = []
  for daynum in range(num_days):
    item = {}
    for x in 'nothing easy medium hard'.split(' '):
      item[x] = 0
    output.append(item)
  for user,daynum_to_difficulty_choices in user_to_daynum_to_difficulty_choices.items():
    has_data = True
    for daynum in range(num_days):
      if not daynum in daynum_to_difficulty_choices:
        has_data = False
        break
    if not has_data:
      continue
    for daynum in range(num_days):
      for difficulty,num_chosen in to_percent_dict(daynum_to_difficulty_choices[daynum]).items():
        output[daynum][difficulty] += num_chosen
      output[daynum] = to_percent_dict(output[daynum])
  return output

def list_of_dictionaries_to_dictionary_with_list_values(dlist):
  output = {}
  keys = dlist[0].keys()
  for k in keys:
    output[k] = []
  for d in dlist:
    for k,v in d.items():
      output[k].append(v)
  return output

def plotline(values, title=''):
  trace = go.Scatter(
    x = list(range(len(values))),
    y=values,
  )
  data = [trace]
  layout = go.Layout(title=title)
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)

def plotlines(dict_to_values, title=''):
  data = []
  for label,values in dict_to_values.items():
    trace = go.Scatter(
      x = list(range(len(values))),
      y=values,
      name=label,
    )
    data.append(trace)
  layout = go.Layout(title=title)
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)

#plotline([3,5,2])
#plotlines({'a': [3,5,2], 'b': [7,7,7]})

In [48]:
def compute_entropy_for_difficulty_selections(difficulty_selection_dict):
  if len(difficulty_selection_dict.keys()) == 0:
    return None
  probs = to_percent_dict(difficulty_selection_dict)
  items_to_sum = []
  for k,prob in probs.items():
    items_to_sum.append(prob * math.log(prob)/math.log(2))
  return -sum(items_to_sum)

def compute_entropy_for_difficulty_selections_for_user(user):
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    return compute_entropy_for_difficulty_selections(difficulty_counts)

def get_entropies_for_user_list(user_list):
  entropies = []
  for user in user_list:
    entropy = compute_entropy_for_difficulty_selections_for_user(user)
    if entropy == None:
      continue
    entropies.append(entropy)
  return entropies

def get_entropies_for_all_users():
  user_list = get_users_with_choose_difficulty()
  return get_entropies_for_user_list(user_list)

def get_entropies_for_all_users_more_than_5():
  user_list = get_users_with_choose_difficulty()
  return get_entropies_for_user_list_with_more_than_5(user_list)

def get_entropies_for_all_users_with_10_first_days():
  user_list = get_users_with_choose_difficulty()
  return get_entropies_for_user_list_with_10_first_days(user_list)

def get_entropies_for_user_list_with_more_than_5(user_list):
  entropies = []
  for user in user_list:
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    total = sum(difficulty_counts.values())
    if total < 5:
      continue
    entropy = compute_entropy_for_difficulty_selections_for_user(user)
    if entropy == None:
      continue
    entropies.append(entropy)
  return entropies


#print(user_list[0])
#print(compute_entropy_for_difficulty_selections({'a': 0.25, 'b': 0.75}))

def get_entropies_for_user_list_with_10_first_days(user_list):
  entropies = []
  user_to_daynum_to_difficulty_choices = get_user_to_daynum_to_difficulty_choices()
  for user in user_list:
    #difficulty_counts = get_choose_difficulty_counts_for_user(user)
    daynum_to_difficulty_choices = user_to_daynum_to_difficulty_choices[user]
    is_valid = True
    for i in range(10):
      if i not in daynum_to_difficulty_choices:
        is_valid = False
        break
    if not is_valid:
      continue
    #total = sum(difficulty_counts.values())
    #if total < 5:
    #  continue
    entropy = compute_entropy_for_difficulty_selections_for_user(user)
    if entropy == None:
      continue
    entropies.append(entropy)
  return entropies


In [46]:
#print(np.mean(entropies))
#print(np.sum(entropies))
entropies = get_entropies_for_all_users_with_10_first_days()
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram)')

NameError: name 'get_user_to_daynum_to_difficulty_choices' is not defined

In [ ]:
#print(np.mean(entropies))
#print(np.sum(entropies))
entropies = get_entropies_for_user_list(difficulty_interface_to_users['time_afford'])
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram), for users with interface=time_afford')

In [ ]:
entropies = get_entropies_for_user_list(difficulty_interface_to_users['this_intervention'])
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram), for users with interface=this_intervention')

In [ ]:
entropies = get_entropies_for_user_list(difficulty_interface_to_users['settings_update'])
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram), for users with interface=settings_update')

In [ ]:
entropies = get_entropies_for_user_list(frequency_of_choose_difficulty_to_users['1.0'])
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram), for users with frequency=1.0')

In [ ]:
entropies = get_entropies_for_user_list(frequency_of_choose_difficulty_to_users['0.5'])
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram), for users with frequency=0.5')

In [ ]:
entropies = get_entropies_for_user_list(frequency_of_choose_difficulty_to_users['0.25'])
print('mean entropy per user', np.mean(entropies))
print('median entropy per user', np.median(entropies))
plothist(entropies, 'Entropies per user, in bits (histogram), for users with frequency=0.25')

In [ ]:
#import moment
#moment.unix(1544559305512.0)
#import arrow
#arrow.get(1544559305512.0 / 1000)

In [ ]:
plotlines(list_of_dictionaries_to_dictionary_with_list_values(get_daynum_to_difficulty_choices_over_n_days(10)), 'Difficulty chosen over first 10 days of install')

In [ ]:
def compute_entropy_over_n_days(num_days):
  daynum_to_difficulty_choices = get_daynum_to_difficulty_choices_over_n_days(num_days)
  output = []
  for daynum in range(num_days):
    difficulty_choices = daynum_to_difficulty_choices[daynum]
    entropy_for_day = compute_entropy_for_difficulty_selections(difficulty_choices)
    output.append(entropy_for_day)
  return output

#print(compute_entropy_over_n_days(10))
plotline(compute_entropy_over_n_days(10), 'Entropy of difficulty choice selections over first 10 days of install')

In [ ]:
plotline(compute_entropy_over_n_days(5), 'Entropy of difficulty choice selections over first 5 days of install')

In [ ]:
user_to_daynum_to_difficulty_choices = get_user_to_daynum_to_difficulty_choices()
print(len(user_to_daynum_to_difficulty_choices.keys()))

In [ ]:
print(get_choose_difficulty_counts_for_user_list(difficulty_interface_to_users['time_afford']))

In [ ]:
print(get_choose_difficulty_counts_for_user_list(difficulty_interface_to_users['this_intervention']))

In [ ]:
print(get_choose_difficulty_counts_for_user_list(difficulty_interface_to_users['settings_update']))

In [ ]:
frequency_of_choose_difficulty_to_users = get_abtest_condition_to_user_list('frequency_of_choose_difficulty')

In [ ]:
frequency_of_choose_difficulty_to_users.keys()

In [ ]:
print(get_choose_difficulty_counts_for_user_list(frequency_of_choose_difficulty_to_users['1.0']))

In [ ]:
print(get_choose_difficulty_counts_for_user_list(frequency_of_choose_difficulty_to_users['0.5']))

In [ ]:
print(get_choose_difficulty_counts_for_user_list(frequency_of_choose_difficulty_to_users['0.25']))

In [ ]:
#print(get_choose_difficulty_counts_for_user_list(frequency_of_choose_difficulty_to_users['0.0']))

In [ ]:
#print('foobar')

In [ ]:
#def get_lifetime_and_whether_attritioned(user):
  

In [ ]:
#choose_difficulty_set = set()
#for x in collection_names:
#  if 'difficulty' in x:
#    choose_difficulty_set.add(x)

In [ ]:
#print(choose_difficulty_set)
#for x in choose_difficulty_set:
  